In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import datetime


# 1. Connection Settings

In [2]:
# Load .env file
load_dotenv()

# Get the EC2 tracking server host from the environment variable
EC2_TRACKING_SERVER_HOST = os.getenv('EC2_TRACKING_SERVER_HOST')
EC2_ENDPOINT = f"http://{EC2_TRACKING_SERVER_HOST}:8000"

# Parameters for the RDS PostgreSQL instance
PG_HOST = os.getenv('PG_HOST')
PG_PORT = os.getenv('PG_PORT')
PG_DATABASE = os.getenv('PG_DATABASE')
PG_USER = os.getenv('PG_USER')
PG_PASSWORD = os.getenv('PG_PASSWORD')

# Create the MySQL database connection string
db_url = f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}'

In [3]:
def ingest_rds(df, table_name, mode):
    engine = create_engine(db_url)
    connection = engine.connect()
    df.to_sql(table_name, engine, if_exists=mode, index=False)
    connection.close()

In [ ]:
df = pd.DataFrame({
    'model_name': ['xgboost-8features-hpt', 'xgboost-8features-hpt-guster3 '],
    'retrained_date': [datetime(2023,11,5,23,0,14) , datetime(2023,11,5,23,2,31)]  # Gets today's date
})

In [ ]:
ingest_rds(df, 'table_update_rewa', 'append')

In [39]:
def select_from_rds(query):
    engine = create_engine(db_url)
    connection = engine.connect()
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [ ]:
select_from_rds('select * from table_update_rewa')

In [40]:
df_m = select_from_rds('select * from measurments_rewa')
df_m.head()

,Time,WindSpeed,WindGust,Temp,WindDir,Baro,Update
0,2021-01-25 00:00:00,6.41,7.19,1.6,257.0,1023.0,2023-10-13 18:27:01
1,2021-01-25 00:10:00,5.05,6.03,1.7,225.0,1023.1,2023-10-13 18:27:01
2,2021-01-25 00:20:00,4.28,6.03,1.7,229.0,1023.3,2023-10-13 18:27:01
3,2021-01-25 00:30:00,3.89,4.67,1.7,235.0,1023.5,2023-10-13 18:27:01
4,2021-01-25 00:40:00,5.64,7.19,2.1,228.0,1023.5,2023-10-13 18:27:01


In [8]:
df_f = select_from_rds('select * from forecast')
df_f.tail()

,Time,Month,Hour,WindForecast,GustForecast,WindDirForecast,Temperature,Precipitation,Cloudcover,Update
12018,2023-10-30 14:00:00,10,14,9.2,17.20,228.5,13.50,0.0,100.0,2023-10-30
12019,2023-10-30 16:00:00,10,16,6.7,11.45,205.0,11.75,0.0,100.0,2023-10-30
12020,2023-10-30 18:00:00,10,18,8.0,18.00,196.5,10.75,0.0,98.0,2023-10-30
12021,2023-10-30 20:00:00,10,20,7.0,17.10,199.0,10.45,0.0,100.0,2023-10-30
12022,2023-10-30 22:00:00,10,22,5.4,13.15,175.0,11.10,0.0,100.0,2023-10-30


In [28]:
df = select_from_rds(query='select * from model_versions')

In [29]:
max_version_gust = df[df['name'] == 'xgboost-8features-hpt-guster3']['version'].max()
max_version_gust

2

In [27]:
max_version = df[df['name'] == 'xgboost-8features-hpt']['version'].max()
max_version

5

In [35]:
df = select_from_rds(query='select * from current_pred_rewa')
df

,Time,Wind,Gust,Direction
0,2023-11-05 00:00:00,10.577015,13.432979,155.5
1,2023-11-05 02:00:00,12.337617,16.567680,155.0
2,2023-11-05 04:00:00,13.835687,17.819675,159.0
3,2023-11-05 06:00:00,13.831645,18.037453,158.0
4,2023-11-05 08:00:00,10.468009,13.563397,162.0
5,2023-11-05 10:00:00,10.813570,14.019954,166.0
6,2023-11-05 12:00:00,9.274166,12.155934,173.0
7,2023-11-05 14:00:00,9.741526,12.875283,170.5
8,2023-11-05 16:00:00,8.706649,11.336173,172.5
9,2023-11-05 18:00:00,7.434165,9.906543,173.0


In [36]:
# Define refined bins and labels for wind direction
bins = np.arange(0, 361, 22.5)
labels = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
            'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW', 'N']

# Bin wind directions with the refined bins and labels
df['Direction'] = pd.cut(df['Direction'] % 360, bins=bins, labels=labels[:-1], include_lowest=True)

# Mapping for direction arrows
arrows = {
    'N': '↓', 'NNE': '↓', 'NE': '↙', 'ENE': '←',
    'E': '←', 'ESE': '←', 'SE': '↗', 'SSE': '↗',
    'S': '↑', 'SSW': '↑', 'SW': '↖', 'WSW': '→',
    'W': '→', 'WNW': '→', 'NW': '↘', 'NNW': '↓'
}

# Add a new column for the arrows
df['Arrow'] = df['Direction'].map(arrows)
df

,Time,Wind,Gust,Direction,Arrow
0,2023-11-05 00:00:00,10.577015,13.432979,SE,↗
1,2023-11-05 02:00:00,12.337617,16.567680,SE,↗
2,2023-11-05 04:00:00,13.835687,17.819675,SSE,↗
3,2023-11-05 06:00:00,13.831645,18.037453,SSE,↗
4,2023-11-05 08:00:00,10.468009,13.563397,SSE,↗
5,2023-11-05 10:00:00,10.813570,14.019954,SSE,↗
6,2023-11-05 12:00:00,9.274166,12.155934,SSE,↗
7,2023-11-05 14:00:00,9.741526,12.875283,SSE,↗
8,2023-11-05 16:00:00,8.706649,11.336173,SSE,↗
9,2023-11-05 18:00:00,7.434165,9.906543,SSE,↗
